In [1]:
import ast
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.metrics import jaccard_score, hamming_loss
from collections import Counter


from transformers import (
    DistilBertModel, 
    DistilBertTokenizerFast,
    get_linear_schedule_with_warmup
)
from tqdm.auto import tqdm


/Users/malgosielska/movie-genre-keyword-prediction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [3]:
data_file = "../movie_dataset_prepared.csv"
df = pd.read_csv(data_file)

In [4]:
df['genres_list'] = df['genres_list'].apply(ast.literal_eval)
df['keywords_list'] = df['keywords_list'].apply(ast.literal_eval)

print(type(df['genres_list'][0]))
df.head()


<class 'list'>


,genres_list,keywords_list,overview
0,"[28, 12, 14, 878]","[9951, 209714]",in the 22nd century a paraplegic marine is dis...
1,"[12, 14, 28]",[179430],captain barbossa long believed to be dead has ...
2,"[28, 12, 80]","[818, 9663]",a cryptic message from bonds past sends him on...
3,"[28, 80, 18, 53]",[9715],following the death of district attorney harve...
4,"[28, 12, 878]","[818, 9951, 10685, 209714]",john carter is a warweary former military capt...


In [5]:
def clean_overview(x):
    if isinstance(x, str):
        return x
    if pd.isna(x):
        return ""
    return str(x)

df['overview'] = df['overview'].apply(clean_overview)


In [6]:
texts = df['overview'].tolist() 
genre_labels = df['genres_list'].tolist()

texts, genre_labels

(['in the 22nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization',
  'captain barbossa long believed to be dead has come back to life and is headed to the edge of the earth with will turner and elizabeth swann but nothing is quite as it seems',
  'a cryptic message from bonds past sends him on a trail to uncover a sinister organization while m battles political forces to keep the secret service alive bond peels back the layers of deceit to reveal the terrible truth behind spectre',
  'following the death of district attorney harvey dent batman assumes responsibility for dents crimes to protect the late attorneys reputation and is subsequently hunted by the gotham city police department eight years later batman encounters the mysterious selina kyle and the villainous bane a new terrorist leader who overwhelms gothams finest the dark knight resurfaces to protect a city that has 

In [7]:
all_genres = [genre for sublist in genre_labels for genre in sublist]
Counter(all_genres)

Counter({18: 2296,
         35: 1722,
         53: 1274,
         28: 1154,
         10749: 894,
         12: 790,
         80: 696,
         878: 535,
         27: 519,
         10751: 513,
         14: 424,
         9648: 348,
         16: 234,
         36: 197,
         10402: 185})

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [9]:
class MovieDataset(Dataset):
    """Movie dataset for multi-label classification"""

    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)
        }
    
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [10]:
class BERTSingleTaskClassifier(nn.Module):
    def __init__(self, n_classes, dropout=0.3, bert_model='distilbert-base-uncased'):
        super(BERTSingleTaskClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        output = self.dropout(pooled_output)
        logits = self.classifier(output)
        return logits

In [11]:
def train_epoch(model, data_loader, criterion, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(data_loader, desc='Training')

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(data_loader)


In [12]:
@torch.no_grad()
def evaluate(model, loader, criterion, device, threshold=0.3):
    model.eval()
    all_preds, all_labels = [], []
    total_loss = 0
    
    for batch in tqdm(loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        preds = (torch.sigmoid(outputs) > threshold).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    metrics = {
        'loss': total_loss / len(loader),
        'f1_micro': f1_score(all_labels, all_preds, average='micro', zero_division=0),
        'f1_macro': f1_score(all_labels, all_preds, average='macro', zero_division=0),
        'precision': precision_score(all_labels, all_preds, average='micro', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='micro', zero_division=0),
        'hamming_accuracy': 1 - hamming_loss(all_labels, all_preds),
        'jaccard_samples': jaccard_score(all_labels, all_preds, average='samples')
    }
    return metrics

In [ ]:
def train_model(
    texts, labels, task='genres',
    epochs=3, batch_size=16, learning_rate=2e-5,
    max_length=256, val_size=0.2, threshold=0.3
):
    mlb = MultiLabelBinarizer()
    labels_binary = mlb.fit_transform(labels)
    n_classes = labels_binary.shape[1]

    counts = labels_binary.sum(axis=0)
    pos_weight = torch.tensor((len(labels_binary) - counts) / counts, dtype=torch.float)

    # Podział
    X_train, X_val, y_train, y_val = train_test_split(
        texts, labels_binary,
        test_size=val_size,
        random_state=42,
        shuffle=True
    )

    # Dataset i DataLoader
    train_dataset = MovieDataset(X_train, y_train, tokenizer, max_length)
    val_dataset = MovieDataset(X_val, y_val, tokenizer, max_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Model
    model = BERTSingleTaskClassifier(n_classes=n_classes).to(device)

    # Strata & optymalizator
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Historia metryk
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_f1_micro': [],
        'val_f1_macro': [],
        'val_precision': [],
        'val_recall': [],
        'val_hamming_accuracy': [],
        'val_jaccard': [],
    }

    best_f1 = 0

    for epoch in range(epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, device)
        val_metrics = evaluate(model, val_loader, criterion, device, threshold)

        # Zapisywanie metryk
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_metrics['loss'])
        history['val_f1_micro'].append(val_metrics['f1_micro'])
        history['val_f1_macro'].append(val_metrics['f1_macro'])
        history['val_precision'].append(val_metrics['precision'])
        history['val_recall'].append(val_metrics['recall'])
        history['val_hamming_accuracy'].append(val_metrics['hamming_accuracy'])
        history['val_jaccard'].append(val_metrics['jaccard_samples'])

        print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f}, Val Loss {val_metrics['loss']:.4f}, F1 Micro {val_metrics['f1_micro']:.4f}")

        # --------- SAVE BEST MODEL ---------
        if val_metrics['f1_micro'] > best_f1:
            best_f1 = val_metrics['f1_micro']
            torch.save(model.state_dict(), f"best_bert_{task}.pth")
            print(f"\n✓ Zapisano najlepszy model! (F1 Micro: {best_f1:.4f})")

    return model, history, mlb, tokenizer


## PARAM GRID

In [14]:
from itertools import product

def grid_search_subset(
    texts, labels,
    param_grid,
    subset_frac=0.1,
    device='cpu',
    task='genres'
):
    texts_subset, _, labels_subset, _ = train_test_split(
        texts, labels,
        test_size=1-subset_frac,
        random_state=42,
        shuffle=True
    )
    
    results = []

    keys = list(param_grid.keys())
    values = list(param_grid.values())

    for combo in product(*values):
        params = dict(zip(keys, combo))
        print("\n" + "="*50)
        print(f"Testing params: {params}")
        print("="*50)
        
        model, history, mlb, _ = train_model(
            texts=texts_subset,
            labels=labels_subset,
            task=task,
            epochs=params['epochs'],
            batch_size=params['batch_size'],
            learning_rate=params['learning_rate'],
            max_length=params['max_length'],
            val_size=0.2, 
            threshold=params['threshold']
        )
        
        f1_micro = history['val_f1_micro'][-1]
        f1_macro = history['val_f1_macro'][-1]
        jaccard = history['val_jaccard'][-1]

        results.append({
            **params,
            'f1_micro': f1_micro,
            'f1_macro': f1_macro,
            'jaccard': jaccard
        })
        
        print(f"→ F1 Micro: {f1_micro:.4f}, F1 Macro: {f1_macro:.4f}, Jaccard: {jaccard:.4f}")
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='f1_micro', ascending=False).reset_index(drop=True)
    
    print("\nBest params based on F1 Micro:")
    display(results_df.head(3))
    
    return results_df

In [15]:
param_grid = {
    'batch_size': [16, 32],
    'learning_rate': [2e-5, 3e-5],
    'max_length': [128, 256],
    'epochs': [3],
    'threshold': [0.25, 0.3, 0.35]
}

In [16]:
results_df = grid_search_subset(
    texts=texts,
    labels=genre_labels,
    param_grid=param_grid,
    subset_frac=0.3,  # używamy 30% danych
    device=device,
    task='genres'
)


Testing params: {'batch_size': 16, 'learning_rate': 2e-05, 'max_length': 128, 'epochs': 3, 'threshold': 0.25}


Evaluating: 100%|██████████| 18/18 [00:01<00:00, 12.77it/s]


Epoch 1: Train Loss 1.1121, Val Loss 1.0053, F1 Micro 0.2852

✓ Zapisano najlepszy model! (F1 Micro: 0.2852)


Evaluating: 100%|██████████| 18/18 [00:01<00:00, 12.44it/s]


Epoch 2: Train Loss 0.9135, Val Loss 0.8819, F1 Micro 0.3207

✓ Zapisano najlepszy model! (F1 Micro: 0.3207)


...

Evaluating: 100%|██████████| 9/9 [00:02<00:00,  3.01it/s]


Epoch 2: Train Loss 0.9920, Val Loss 0.9375, F1 Micro 0.3635

✓ Zapisano najlepszy model! (F1 Micro: 0.3635)


Evaluating: 100%|██████████| 9/9 [00:03<00:00,  2.94it/s]


Epoch 3: Train Loss 0.8678, Val Loss 0.8898, F1 Micro 0.4002

✓ Zapisano najlepszy model! (F1 Micro: 0.4002)
→ F1 Micro: 0.4002, F1 Macro: 0.3557, Jaccard: 0.2600

Best params based on F1 Micro:


,batch_size,learning_rate,max_length,epochs,threshold,f1_micro,f1_macro,jaccard
0,16,0.00003,256,3,0.35,0.481022,0.424053,0.326835
1,16,0.00003,128,3,0.35,0.474436,0.421071,0.323235
2,16,0.00003,128,3,0.30,0.441681,0.391938,0.295145


In [18]:
grid_results_path = "param_grid/grid_search_results_genres.csv"
results_df.to_csv(grid_results_path, index=False)

print(f"Wyniki grid search zapisane do: {grid_results_path}")

Wyniki grid search zapisane do: param_grid/grid_search_results_genres.csv


## 5-FOLD VALIDATION

In [ ]:
from sklearn.model_selection import KFold

def train_5fold(
    texts, labels,
    n_splits=5,
    batch_size=32,
    learning_rate=2e-5,
    max_length=256,
    epochs=5,
    threshold=0.3,
    device='cpu',
    task='genres'
):
    mlb = MultiLabelBinarizer()
    labels_binary = mlb.fit_transform(labels)
    n_classes = labels_binary.shape[1]

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = []
    fold_best_metrics = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(texts)):
        X_train = [texts[i] for i in train_idx]
        y_train = labels_binary[train_idx]
        X_val = [texts[i] for i in val_idx]
        y_val = labels_binary[val_idx]

        train_dataset = MovieDataset(X_train, y_train, tokenizer, max_length)
        val_dataset = MovieDataset(X_val, y_val, tokenizer, max_length)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        model = BERTSingleTaskClassifier(n_classes=n_classes).to(device)

        counts = y_train.sum(axis=0)
        pos_weight = torch.tensor((len(y_train) - counts) / counts, dtype=torch.float).to(device)

        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
        optimizer = AdamW(model.parameters(), lr=learning_rate)

        total_steps = len(train_loader) * epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=total_steps
        )

        best_f1 = 0
        best_fold_metrics = None

        history = {
            'train_loss': [],
            'val_loss': [],
            'val_f1_micro': [],
            'val_f1_macro': [],
            'val_precision': [],
            'val_recall': [],
            'val_hamming_accuracy': [],
            'val_jaccard': [],
        }

        for epoch in range(epochs):
            train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, device)
            val_metrics = evaluate(model, val_loader, criterion, device, threshold)

            # Historia
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_metrics['loss'])
            history['val_f1_micro'].append(val_metrics['f1_micro'])
            history['val_f1_macro'].append(val_metrics['f1_macro'])
            history['val_precision'].append(val_metrics['precision'])
            history['val_recall'].append(val_metrics['recall'])
            history['val_hamming_accuracy'].append(val_metrics['hamming_accuracy'])
            history['val_jaccard'].append(val_metrics['jaccard_samples'])

            print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f}, Val Loss {val_metrics['loss']:.4f}, F1 Micro {val_metrics['f1_micro']:.4f}")

            if val_metrics['f1_micro'] > best_f1:
                best_f1 = val_metrics['f1_micro']
                torch.save(model.state_dict(), f"best_bert_{task}_fold{fold+1}.pth")
                best_fold_metrics = val_metrics.copy()  # zapisujemy metryki najlepszego modelu

        fold_metrics.append({k: v[-1] for k, v in history.items() if k.startswith('val')})
        fold_best_metrics.append(best_fold_metrics)

        print(f"Fold {fold+1} metrics: {fold_metrics[-1]}")

    metrics_df = pd.DataFrame(fold_metrics)
    mean_metrics = metrics_df.mean()
    std_metrics = metrics_df.std()
    print("\n=== 5-FOLD CROSS-VALIDATION SUMMARY ===")
    print("Mean metrics:\n", mean_metrics)
    print("Std metrics:\n", std_metrics)

    return metrics_df, mean_metrics, std_metrics, best_fold_metrics, mlb

best_params = {
    'batch_size': 16,
    'learning_rate': 3e-5,
    'max_length': 256,
    'epochs': 7,
    'threshold': 0.35
}

metrics_df, mean_metrics, std_metrics, best_fold_metrics, mlb_genres = train_5fold(
    texts=texts,
    labels=genre_labels,
    n_splits=5,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    max_length=best_params['max_length'],
    epochs=best_params['epochs'],
    threshold=best_params['threshold'],
    device=device,
    task='genres'
)


Evaluating: 100%|██████████| 59/59 [00:10<00:00,  5.81it/s]


Epoch 1: Train Loss 0.8777, Val Loss 0.7463, F1 Micro 0.5002


Evaluating: 100%|██████████| 59/59 [00:10<00:00,  5.81it/s]


Epoch 2: Train Loss 0.6347, Val Loss 0.6972, F1 Micro 0.5628


Evaluating: 100%|██████████| 59/59 [00:10<00:00,  5.81it/s]


...

Epoch 5: Train Loss 0.3693, Val Loss 0.7462, F1 Micro 0.6283


Evaluating: 100%|██████████| 59/59 [01:00<00:00,  1.03s/it]


Epoch 6: Train Loss 0.3257, Val Loss 0.7728, F1 Micro 0.6292


Evaluating: 100%|██████████| 59/59 [02:21<00:00,  2.40s/it]


Epoch 7: Train Loss 0.3016, Val Loss 0.7817, F1 Micro 0.6312
Fold 5 metrics: {'val_loss': 0.7817470284841829, 'val_f1_micro': 0.631197496293856, 'val_f1_macro': 0.5791867436119821, 'val_precision': 0.5164420485175202, 'val_recall': 0.8115205421431597, 'val_hamming_accuracy': 0.8405270655270656, 'val_jaccard': 0.4913317392484059}

=== 5-FOLD CROSS-VALIDATION SUMMARY ===
Mean metrics:
 val_loss                0.778350
val_f1_micro            0.631989
val_f1_macro            0.584318
val_precision           0.514785
val_recall              0.818356
val_hamming_accuracy    0.840146
val_jaccard             0.493978
dtype: float64
Std metrics:
 val_loss                0.050710
val_f1_micro            0.007342
val_f1_macro            0.011967
val_precision           0.006344
val_recall              0.011486
val_hamming_accuracy    0.004423
val_jaccard             0.007673
dtype: float64


In [ ]:
print("\nMetryki dla poszczególnych foldów:")
display(metrics_df)

print("\nŚrednie metryki po 5-fold:")
display(mean_metrics)

print("\nOdchylenie standardowe po 5-fold:")
display(std_metrics)


Metryki dla poszczególnych foldów:


,val_loss,val_f1_micro,val_f1_macro,val_precision,val_recall,val_hamming_accuracy,val_jaccard
0,0.760987,0.625835,0.588622,0.507262,0.816752,0.836571,0.489652
1,0.864157,0.624696,0.570444,0.510881,0.803758,0.835432,0.485960
2,0.738585,0.642466,0.602332,0.524067,0.829979,0.846531,0.505542
3,0.746272,0.635753,0.581004,0.515272,0.829769,0.841667,0.497407
4,0.781747,0.631197,0.579187,0.516442,0.811521,0.840527,0.491332



Średnie metryki po 5-fold:


val_loss                0.778350
val_f1_micro            0.631989
val_f1_macro            0.584318
val_precision           0.514785
val_recall              0.818356
val_hamming_accuracy    0.840146
val_jaccard             0.493978
dtype: float64


Odchylenie standardowe po 5-fold:


val_loss                0.050710
val_f1_micro            0.007342
val_f1_macro            0.011967
val_precision           0.006344
val_recall              0.011486
val_hamming_accuracy    0.004423
val_jaccard             0.007673
dtype: float64

## GENRES PREDICTION

In [37]:
def predict_genres(text, model, tokenizer, mlb, device, threshold=0.5, top_k=5):
    model.eval()
    
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    
    # Get top-k predictions
    top_indices = np.argsort(probs)[::-1][:top_k]
    top_keywords = [(mlb.classes_[i], probs[i]) for i in top_indices if probs[i] > threshold]
    
    return top_keywords


In [38]:
genres_map_df = pd.read_csv("../genres_id_to_name_mapping.csv")
id_to_genre = dict(zip(genres_map_df["id"], genres_map_df["name"]))

In [39]:
def ids_to_genre_names(ids, mapping):
    return [mapping[i] for i in ids if i in mapping]

In [42]:
def show_sample_predictions_genres(
    texts,
    labels,
    model_path,
    tokenizer,
    id_to_genre,
    device,
    num_samples=5
):
    mlb = MultiLabelBinarizer()
    labels_binary = mlb.fit_transform(labels)
    n_classes = labels_binary.shape[1]

    # Load model
    n_classes = len(mlb.classes_)
    model = BERTSingleTaskClassifier(n_classes=n_classes).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Random sample indices
    import random
    indices = random.sample(range(len(texts)), num_samples)

    for idx in indices:
        sample_text = texts[idx]
        
        # True labels → names
        true_ids = mlb.inverse_transform(labels_binary[idx:idx+1])[0]
        true_genres = ids_to_genre_names(true_ids, id_to_genre)

        # Predict
        predicted_raw = predict_genres(sample_text, model, tokenizer, mlb, device)

        # Convert predicted IDs → names
        predicted_genres = [
            (id_to_genre.get(gid, f"UNKNOWN_{gid}"), prob)
            for gid, prob in predicted_raw
        ]

        print("\n" + "-" * 60)
        print(f"SAMPLE {idx}")

        print(f"Text:\n{sample_text[:350]}...\n")
        print(f"Actual genres: {true_genres}\n")

        print("Predicted genres (probabilities):")
        for gname, prob in predicted_genres:
            print(f"  - {gname}: {prob:.4f}")


In [43]:
show_sample_predictions_genres(
    texts=texts,
    labels=genre_labels,
    model_path="best_bert_genres_fold1.pth",
    tokenizer=tokenizer,
    id_to_genre=id_to_genre,
    device=device,
    num_samples=5
)


------------------------------------------------------------
SAMPLE 3984
Text:
andrew returns to his hometown for the funeral of his mother a journey that reconnects him with past friends the trip coincides with his decision to stop taking his powerful antidepressants a chance meeting with sam a girl also suffering from various maladies opens up the possibility of rekindling emotional attachments confronting his psychologist ...

Actual genres: ['Drama', 'Comedy', 'Romance']

Predicted genres (probabilities):
  - Drama: 0.8802
  - Romance: 0.7210

------------------------------------------------------------
SAMPLE 2245
Text:
freddie steinmark an underdog on the gridiron faces the toughest challenge of his life after leading his team to a championship season...

Actual genres: ['Drama']

Predicted genres (probabilities):
  - Drama: 0.9039

------------------------------------------------------------
SAMPLE 94
Text:
light years from earth 26 years after being abducted peter quill finds 